Часть 0: Знакомство с scikit-image
======

**scikit-image** - библиотека обработки изображений с открытым исходным кодом для языка программирования Python.

### Краткий ликбез в SciPy
**SciPy** - это экосистема программного обеспечения с открытым исходным кодом для математических, научных и инженерных расчетов, основанная на Python.

Она состоит из следующих пакетов:

    1. NumPy - основной пакет N-мерных массивов
    2. SciPy library - фундаментальная библиотека для научных вычислений
    3. Matplotlib - комплексное 2D-построение
    4. IPython - расширенная интерактивная консоль
    5. Sympy - символьные вычисления
    6. pandas - структуры данных и анализ
    
### SciKits
SciKits (сокращение от SciPy Toolkits), это дополнительные пакеты для SciPy, размещеные и разработаные отдельно и независимо от основного дистрибутива SciPy. 

Пакеты упаковываются как набор инструментов, когда: 
* Пакет считается слишком специализированным, чтобы жить в самом SciPy 
* Пакет имеет лицензию GPL (или аналогичную), которая несовместима с лицензией BSD от SciPy 
* Пакет должен быть включен в SciPy, но разработка все еще продолжается.


### scikit-image

scikit-image - это библиотека с открытым исходным кодом для обработки изображений, разработанная для языка Python.

Она содержит в себе большое количество готовых алгоритмов обработки изображений.

scikit-image является SciKit и расширяет scipy.ndimage для обеспечения универсального набора процедур обработки изображений.

### Установка

Для установки можно использовать pip:

```
pip install -U scikit-image
```

Или конду:

```
conda install -c conda-forge scikit-image
```

[Или собрать проект из исходников](https://scikit-image.org/docs/dev/install.html)


 # Часть 1: Игры с numpy

В scikit-image изображения представлены ввиде массивов NumPy размерностей N * M или N * M * 3 или N * M * 4.

Давайте это проверим!

В начале достанем из интернетов произвольную картинку для экспериментов:

In [2]:
from IPython.display import Image
Image(url='images/pikachu.png')

Отключим warning'и:

In [3]:
import warnings
warnings.filterwarnings("ignore")

Подключим библиотеки и загрузим нашу картинку:

In [5]:
import numpy as np
from  skimage import io
image = io.imread('images/pikachu.png')

Убедимся, что это numpy

In [6]:
print('type:', type(image))
print('shape:', image.shape)
print('min:', image.min())
print('mean:', image.mean())
print('max:', image.max())

type: <class 'numpy.ndarray'>
shape: (524, 720, 4)
min: 0
mean: 146.54529328350296
max: 255


Доберемся до конкретного пикселя

In [8]:
print(image[100, 100])

[  1 149  54 255]


Поиграем со срезами

In [10]:
image[345:370, 375:395] = [10, 10, 10, 255]
io.imsave('images/h.png', image)
Image(url='images/h.png')

А теперь с масками

In [11]:
image = io.imread('images/pikachu.png')
mask = np.logical_and(np.logical_and(image[:, :, 0] > 150, image[:, :, 1] > 150), image[:, :, 2] < 150)
image[mask] = [128, 166, 255, 255]
io.imsave('images/g.png', image)
Image(url='images/g.png')

Выглядит отлично. Из этого можно заключить, что это настоящий NumPy

# Часть 2: В которой мы наконец-то занимаемся чем-то полезным

Все любят Стивена Вольфрама, а Стивен Вольфрам в свою очередь любит клеточные автоматы.

Почему бы нам не написать один из самых популярных и знаменитых клеточных автоматов? (Да-да я про игру Жизнь)

Эти функции будут заниматся масштабированием нашей вселенной. Не очень удобно смореть на клетки размером в 1 пиксель.

In [12]:
def subscale(image, x, y, value, coff):
	for w in range(coff):
		for h in range(coff):
			image[x + w, y + h] = value

def scale(image, coff):
	shape = image.shape
	newimage = np.random.randint(0, 2, size=(image.shape[0] * coff, image.shape[1] * coff))

	for i in range(shape[0]):
		for j in range(shape[1]):
			subscale(newimage, i * coff, j * coff, image[i, j], coff)

	return newimage

Эта связка функций будет отвечать за эволюцию нашей вселенной. Иными словами она отвечает за генерацию состояния следующего шага.

In [13]:
def next(x, y, shape):
	return x - shape[0] if(x >= shape[0]) else x, y - shape[1] if(y >= shape[1]) else y

def evolve(data):
	base = np.copy(data)
	field = np.copy(data)
	shape = field.shape
	for i in range(shape[0]):
		for j in range(shape[1]):
			friendly_neighborhood = base[i - 1, j - 1] + base[i, j - 1] + base[next(i + 1, j - 1, shape)] + base[i - 1, j] + base[next(i + 1, j, shape)] + base[next(i - 1, j + 1, shape)] + base[next(i, j + 1, shape)] + base[next(i + 1, j + 1, shape)]
			if(base[i, j]):
				field[i, j] = int(friendly_neighborhood == 2 or friendly_neighborhood == 3)
			else:
				field[i, j] = int(friendly_neighborhood == 3)
	return field

Простые обертки, которые помогут нам не сильно заморачиваться с кодом:

In [14]:
def generateStateOfLive(base, iterations):
	states = [base]
	for i in range(iterations):
		states.append(evolve(states[-1]))

	return states

def generateRandomStateOfLive(size, iterations):
	field = np.random.randint(0, 2, size=size)
	return generateStateOfLive(field, iterations)

Ура, начинаем играть :)

In [16]:
import os

# Играем в жизнь
states = generateRandomStateOfLive(size=(10, 10), iterations=10)

# Сохраняем
if not os.path.exists('images/kek'):
	os.makedirs('images/kek')

for i in range(len(states)):
	io.imsave('images/kek/{:04d}.png'.format(i + 1), scale(states[i] * 255, 50))

In [17]:
from IPython.display import Image
Image(url='images/kek/0001.png')

А вот этот кусочек кода сделает глайдер)

In [18]:
field = np.zeros((10, 10))
field[0, 1] = 1
field[1, 2] = 1
field[2, 0] = 1
field[2, 1] = 1
field[2, 2] = 1
states = generateStateOfLive(field, iterations=10)

# Часть 3: Анализируем производительность

Пока мы не сильно отошли от игры в жизнь давайте возьмем ее за основу для анализа производительности этой библиотеки. 
Как ее конечные пользователи мы должны знать как хорошо она справляется со своими обязаностями.
Конечно, основной смысл этой библиотеки это алгоритмы (и мы до них еще доберемся!), но алгоритмы сложно анализировать в контексте библиотеки и в любом случае они будут реализованы через элементарные функции библиотеки.
